In [1]:
import  pandas as pd
import numpy as np

In [2]:
file10 = "csv-origins/GE2010_byCounty.csv"
ge10 = pd.read_csv(file10)

# Cleaning 2010 Results

In [3]:
k10, n10 = ge10.keys(), ge10.size

In [4]:
#check out the value counts for each column
valno10 = dict.fromkeys(k10)
for k in k10:
    valno10[k] = getattr(ge10,k).value_counts()

In [5]:
#these columns are not useful
dropvar10 = ["CanAffilCommit", "Election", "VoteFor", "PartyAbbrev","OfficeSequence"]

In [6]:
#so lets drop them
ge10.drop(columns=dropvar10,inplace=True)

In [7]:
nocandbool10 = list(ge10.CanLastName == "NO CANDIDATE")
nocandindx10 = ge10[nocandbool10].index
ge10.drop(index=nocandindx10,inplace=True)

In [8]:
ge10.loc[[" - YES" in x for x in ge10["CanLastName"]], "PartyName"] = "NON-PARTISAN"
ge10.loc[[" - NO" in x for x in ge10["CanLastName"]], "PartyName"] = "NON-PARTISAN"

In [9]:
for p in ge10.CanLastName:
    for l in p:
        if l != l.capitalize():
            capl = p.replace(l, l.capitalize())
            ge10.replace(p,capl,inplace=True)

In [10]:
nulloc10 = ge10.isnull().values
nansdf10 = ge10[nulloc10].copy()
nans = nansdf10.PartyName.values[0]
nancandlist10 = nansdf10[['CandidateID','CanFirstName','CanLastName','OfficeName']].drop_duplicates(subset="CandidateID")
nancandlist10 = nancandlist10.sort_values(by="CanLastName")
nancandlist10 = nancandlist10.values

In [11]:
namelist10 = []
for l in nancandlist10:
    namelist10.append("".join(str(l)))

In [12]:
with open("nancandlist10.txt", 'w') as output:
    for row in namelist10:
        output.write(str(row) + '\n')

In [13]:
write_ins10 = list(nansdf10.drop_duplicates(subset="CandidateID")["CanLastName"].values)
ge10[[x in write_ins10 for x in ge10.CanLastName]] = ge10[[x in write_ins10 for x in ge10.CanLastName]].replace(nans,"WRITE IN")

In [14]:
candidates10 = ge10[["CandidateID","CanFirstName", "CanLastName","PartyName","OfficeName"]].copy()
candidates10.drop_duplicates(inplace=True,subset="CandidateID")
candidates10.set_index(candidates10.CandidateID,inplace=True)
candidates10.drop(columns="CandidateID",inplace=True)
candidates10.sort_values(by="CandidateID",inplace=True)

In [15]:
runningno10 = ge10.OfficeName.value_counts()
rndict10 = dict(runningno10)
rnlist10 = []
for o in ge10.OfficeName:
    rnlist10.append(rndict10[o])
NumberRunning10 = pd.Series(rnlist10,name="NumberRunning")

In [16]:
bg10 = ge10[["OfficeName","BallotGroup"]].copy()
bg10 = bg10.merge(NumberRunning10,left_index=True, right_index=True)
bg10.drop_duplicates(subset="OfficeName",inplace=True,ignore_index=True)

In [17]:
bg10count = bg10.groupby("BallotGroup")
bg10dict = dict.fromkeys(bg10.BallotGroup.drop_duplicates())

In [18]:
for bg in bg10.BallotGroup.drop_duplicates():
    nrsum = bg10count.get_group(bg).NumberRunning.sum()
    bg10dict[bg] = nrsum

In [19]:
bg10list = []
for bg in bg10.BallotGroup:
    bg10list.append(bg10dict[bg])
bg10ser = pd.Series(bg10list, index=bg10.index, name="BallotGrpCount")

In [20]:
bg10 = bg10.merge(bg10ser,left_index=True,right_index=True)

In [21]:
indx = pd.MultiIndex.from_frame(bg10[["BallotGroup","OfficeName"]])
bg10.set_index(indx,inplace=True)

In [22]:
bg10.sort_values(by=["BallotGrpCount","NumberRunning"], ascending=[False,False],inplace=True)

In [23]:
bg10.drop(columns=["BallotGroup","OfficeName","BallotGrpCount"],inplace=True)

In [24]:
ballotgroups10 = bg10.copy()

In [25]:
votes10 = ge10[["CandidateID","County","Votes"]].copy()
indx2 = pd.MultiIndex.from_frame(ge10[["CandidateID","County"]])
votes10.set_index(indx2,inplace=True)
votes10.drop(columns=["CandidateID","County"],inplace=True)
votes10.sort_values(by=['CandidateID','Votes'],ascending=[True,False],inplace=True)

In [26]:
candidates10.head()

,CanFirstName,CanLastName,PartyName,OfficeName
CandidateID,,,,
15694,RICH,BRAUER,REPUBLICAN,100TH REPRESENTATIVE
15695,BOB,FLIDER,DEMOCRATIC,101ST REPRESENTATIVE
15697,MARK STEVEN,KIRK,REPUBLICAN,UNITED STATES SENATOR
15698,"ALEXANDER ""ALEXI""",GIANNOULIAS,DEMOCRATIC,UNITED STATES SENATOR
15699,LeALAN M.,JONES,GREEN,UNITED STATES SENATOR


In [27]:
ballotgroups10.head()

NumberRunning
BallotGroup OfficeName                                     
B           GOVERNOR AND LIEUTENANT GOVERNOR            777
            ATTORNEY GENERAL                            408
            COMPTROLLER                                 408
            TREASURER                                   408
            SECRETARY OF STATE                          306

In [28]:
votes10.head()

Votes
CandidateID County         
15694       SANGAMON  32318
            MENARD     4085
            LOGAN      4071
15695       MACON     12464
            MOULTRIE   1893

# Export tables to csvs


In [29]:
candidates10.to_csv("csv-clean/candidates10.csv",index_label="CandidateID")
ballotgroups10.to_csv("csv-clean/ballotgroups10.csv",index_label=["BallotGroup","OfficeName"])
votes10.to_csv("csv-clean/votes10.csv",index_label=["CandidateID","County"])

# Cleaning 2014 Results

In [30]:
file14 = "csv-origins/GE2014_byCounty.csv"
ge14 = pd.read_csv(file14)

In [31]:
#start the same as with 2010
k14, n14 = ge14.keys(), ge14.size

In [32]:
valno14 = dict.fromkeys(k14)
for k in k14:
    valno14[k] = getattr(ge14,k).value_counts()

In [33]:
#drop the same as in 2010, also new column ID
dropvar14 = dropvar10 + ['ID']

In [34]:
ge14.drop(columns=dropvar14,inplace=True)

In [35]:
ge14.loc[[" - YES" in x for x in ge14["CanLastName"]], "PartyName"] = "NON-PARTISAN"
ge14.loc[[" - NO" in x for x in ge14["CanLastName"]], "PartyName"] = "NON-PARTISAN"

In [36]:
ge14.loc[(ge14.BallotGroup == "1A").values,"CanFirstName"] = "AMENDMENT"
ge14.loc[(ge14.BallotGroup == "1A").values,"PartyName"] = "NON-PARTISAN"

In [37]:
ge14.loc[(ge14.BallotGroup == "2A").values,"CanFirstName"] = "PROPOSITION"
ge14.loc[(ge14.BallotGroup == "2A").values,"PartyName"] = "NON-PARTISAN"

In [38]:
nulloc14 = ge14.isnull().values
nansdf14 = ge14[nulloc14]
nancandlist14 = nansdf14[['CandidateID','CanFirstName','CanLastName','OfficeName']].drop_duplicates(subset="CandidateID")
nancandlist14 = nancandlist14.sort_values(by="OfficeName")
nancandlist14 = nancandlist14.values

In [39]:
namelist14 = []
for l in nancandlist14:
    namelist14.append("".join(str(l)))

In [40]:
with open("nancandlist14.txt", 'w') as output:
    for row in namelist14:
        output.write(str(row) + '\n')

In [41]:
write_ins14 = list(nansdf14.drop_duplicates(subset="CandidateID")["CanLastName"].values)
ge14[[x in write_ins14 for x in ge14.CanLastName]] = ge14[[x in write_ins14 for x in ge14.CanLastName]].replace(nans,"WRITE IN")

## Instead of making one giant multiindex, I thought it made sense to split the dataset into three tables to take advantage of primary and foreign keys.
## Frame "ge14" will split into "candidates14", "ballotgroups14", and "votes14"
### Frame "ballotgroup14" contains a new column, "NumberRunning", counting how many candidates there are for each office.

In [42]:
candidates14 = ge14[["CandidateID","CanFirstName", "CanLastName","PartyName","OfficeName"]].copy()
candidates14.drop_duplicates(inplace=True,subset="CandidateID")
candidates14.set_index(candidates14.CandidateID,inplace=True)
candidates14.drop(columns="CandidateID",inplace=True)
candidates14.sort_values(by="CandidateID",inplace=True)

In [43]:
runningno14 = ge14.OfficeName.value_counts()
rndict14 = dict(runningno14)
rnlist14 = []
for o in ge14.OfficeName:
    rnlist14.append(rndict14[o])
NumberRunning14 = pd.Series(rnlist14,name="NumberRunning")

In [44]:
bg14 = ge14[["OfficeName","BallotGroup"]].copy()
bg14 = bg14.merge(NumberRunning14,left_index=True, right_index=True)
bg14.drop_duplicates(subset="OfficeName",inplace=True,ignore_index=True)

In [45]:
bg14count = bg14.groupby("BallotGroup")
bg14dict = dict.fromkeys(bg14.BallotGroup.drop_duplicates())

for bg in bg14.BallotGroup.drop_duplicates():
    nrsum = bg14count.get_group(bg).NumberRunning.sum()
    bg14dict[bg] = nrsum

bg14list = []
for bg in bg14.BallotGroup:
    bg14list.append(bg14dict[bg])
bg14ser = pd.Series(bg14list, index=bg14.index, name="BallotGrpCount")

In [46]:
bg14 = bg14.merge(bg14ser,left_index=True,right_index=True)

In [47]:
indx = pd.MultiIndex.from_frame(bg14[["BallotGroup","OfficeName"]])
bg14.set_index(indx,inplace=True)

In [48]:
bg14.sort_values(by=["BallotGrpCount","NumberRunning"], ascending=[False,False],inplace=True)

In [49]:
bg14.drop(columns=["BallotGroup","OfficeName","BallotGrpCount"],inplace=True)

In [50]:
ballotgroups14 = bg14.copy()

In [51]:
votes14 = ge14[["CandidateID","County","Votes"]].copy()
indx2 = pd.MultiIndex.from_frame(ge14[["CandidateID","County"]])
votes14.set_index(indx2,inplace=True)
votes14.drop(columns=["CandidateID","County"],inplace=True)
votes14.sort_values(by=['CandidateID','Votes'],ascending=[True,False],inplace=True)

In [52]:
candidates14.head()

,CanFirstName,CanLastName,PartyName,OfficeName
CandidateID,,,,
19443,CHAPIN,ROSE,REPUBLICAN,51ST SENATE
19444,CHRIS E.,REIF,REPUBLICAN,7TH CIRCUIT - MITCHELL VACANCY
19445,MIKE,ATTERBERRY,REPUBLICAN,8TH CIRCUIT - POPE VACANCY
19446,THOMAS B.,EWING,DEMOCRATIC,9TH CIRCUIT - DANNER VACANCY
19447,DAVE,CARLSON,REPUBLICAN,12TH CIRCUIT - 1ST SUBCIRCUIT - A VACANCY


In [53]:
ballotgroups14.head()

NumberRunning
BallotGroup OfficeName                                     
B           GOVERNOR AND LIEUTENANT GOVERNOR            761
            COMPTROLLER                                 403
            SECRETARY OF STATE                          334
            ATTORNEY GENERAL                            306
            TREASURER                                   306

In [54]:
votes14.head()

Votes
CandidateID County          
19443       MACON      16658
            CHAMPAIGN  15501
            SHELBY      7116
            PIATT       6443
            DOUGLAS     5623

# Export tables to csvs

In [55]:
candidates14.to_csv("csv-clean/candidates14.csv",index_label="CandidateID")
ballotgroups14.to_csv("csv-clean/ballotgroups14.csv",index_label=["BallotGroup","OfficeName"])
votes14.to_csv("csv-clean/votes14.csv",index_label=["CandidateID","County"])

# Will continue in the same way with GE18

In [56]:
file18 = "csv-origins/GE2018_byCounty.csv"
ge18 = pd.read_csv(file18)

In [57]:
k18, n18 = ge18.keys(), ge18.size

In [58]:
valno18 = dict.fromkeys(k18)
for k in k18:
    valno18[k] = getattr(ge18,k).value_counts()

In [59]:
dropvar18 = dropvar14

In [60]:
ge18.drop(columns=dropvar18,inplace=True)

In [61]:
ge18.loc[["- YES" in x for x in ge18["CanLastName"]], "PartyName"] = "NON-PARTISAN"
ge18.loc[["- NO" in x for x in ge18["CanLastName"]], "PartyName"] = "NON-PARTISAN"
ge18.loc[(ge18.CandidateID == 24626).values, "PartyName"] = "NON-PARTISAN"


In [62]:
ge18.loc[(ge18.BallotGroup == "1A").values,"CanFirstName"] = "AMENDMENT"
ge18.loc[(ge18.BallotGroup == "1A").values,"PartyName"] = "NON-PARTISAN"


In [63]:
nulloc18 = ge18.isnull().values
nansdf18 = ge18[nulloc18]
nancandlist18 = nansdf18[['CanFirstName','CanLastName','OfficeName','CandidateID']].drop_duplicates(subset='CandidateID')
nancandlist18 = nancandlist18.sort_values(by="OfficeName")
nancandlist18 = nancandlist18.values

In [64]:
namelist18 = []
for l in nancandlist18:
    namelist18.append("".join(str(l)))

In [65]:
with open("nancandlist18.txt", 'w') as output:
    for row in namelist18:
        output.write(str(row) + '\n')

In [66]:
write_ins18 = list(nansdf18.drop_duplicates(subset="CandidateID")["CanLastName"].values)
ge18[[x in write_ins18 for x in ge18.CanLastName]] = ge18[[x in write_ins18 for x in ge18.CanLastName]].replace(nans,"WRITE IN")

In [67]:
candidates18 = ge18[["CandidateID","CanFirstName", "CanLastName","PartyName","OfficeName"]].copy()
candidates18.drop_duplicates(inplace=True,subset="CandidateID")
candidates18.set_index(candidates18.CandidateID,inplace=True)
candidates18.drop(columns="CandidateID",inplace=True)
candidates18.sort_values(by="CandidateID",inplace=True)

In [68]:
runningno18 = ge18.OfficeName.value_counts()
rndict18 = dict(runningno18)
rnlist18 = []
for o in ge18.OfficeName:
    rnlist18.append(rndict18[o])
NumberRunning18 = pd.Series(rnlist18,name="NumberRunning")

In [69]:
bg18 = ge18[["OfficeName","BallotGroup"]].copy()
bg18 = bg18.merge(NumberRunning18,left_index=True, right_index=True)
bg18.drop_duplicates(subset="OfficeName",inplace=True,ignore_index=True)

In [70]:
bg18count = bg18.groupby("BallotGroup")
bg18dict = dict.fromkeys(bg18.BallotGroup.drop_duplicates())

In [71]:
for bg in bg18.BallotGroup.drop_duplicates():
    nrsum = bg18count.get_group(bg).NumberRunning.sum()
    bg18dict[bg] = nrsum

In [72]:
bg18list = []
for bg in bg18.BallotGroup:
    bg18list.append(bg18dict[bg])
bg18ser = pd.Series(bg18list, index=bg18.index, name="BallotGrpCount")

In [73]:
bg18 = bg18.merge(bg18ser,left_index=True,right_index=True)

In [74]:
indx = pd.MultiIndex.from_frame(bg18[["BallotGroup","OfficeName"]])
bg18.set_index(indx,inplace=True)

In [75]:
bg18.sort_values(by=["BallotGrpCount","NumberRunning"], ascending=[False,False],inplace=True)

In [76]:
bg18.drop(columns=["BallotGroup","OfficeName","BallotGrpCount"],inplace=True)

In [77]:
ballotgroups18 = bg18.copy()

In [78]:
votes18 = ge18[["CandidateID","County","Votes"]].copy()
indx2 = pd.MultiIndex.from_frame(ge18[["CandidateID","County"]])
votes18.set_index(indx2,inplace=True)
votes18.drop(columns=["CandidateID","County"],inplace=True)
votes18.sort_values(by=['CandidateID','Votes'],ascending=[True,False],inplace=True)

In [79]:
candidates18

,CanFirstName,CanLastName,PartyName,OfficeName
CandidateID,,,,
24167,TOYA T.,HARVEY,DEMOCRATIC,COOK - 2ND SUBCIRCUIT - RHODES VACANCY
24168,IESHIA,GRAY,DEMOCRATIC,"COOK - 2ND SUBCIRCUIT - TURNER, JR. VACANCY"
24169,DEBRA A.,SEATON,DEMOCRATIC,COOK - 2ND SUBCIRCUIT - WILLIS VACANCY
24170,KEVIN PATRICK,CUNNINGHAM,DEMOCRATIC,COOK - 3RD SUBCIRCUIT - DELEHANTY VACANCY
24171,GARY WILLIAM,SEYRING,REPUBLICAN,COOK - 13TH SUBCIRCUIT - CRANE VACANCY
...,...,...,...,...
24878,MARY ARLINE,VANN-METCALF,WRITE IN,17TH SENATE
24879,JUSTIN,TUCKER,WRITE IN,4TH REPRESENTATIVE
24880,DENISE ANITA,HILL,WRITE IN,25TH REPRESENTATIVE


In [80]:
ballotgroups18.head()

NumberRunning
BallotGroup OfficeName                                     
B           GOVERNOR AND LIEUTENANT GOVERNOR            877
            COMPTROLLER                                 307
            TREASURER                                   307
            ATTORNEY GENERAL                            306
            SECRETARY OF STATE                          306

In [81]:
votes18.head()

,,Votes
CandidateID,County,
24167,COOK,77612
24168,COOK,76769
24169,COOK,77506
24170,COOK,84717
24171,COOK,52737



# Export tables to csvs


In [82]:
candidates18.to_csv("csv-clean/candidates18.csv",index_label="CandidateID")
ballotgroups18.to_csv("csv-clean/ballotgroups18.csv",index_label=["BallotGroup","OfficeName"])
votes18.to_csv("csv-clean/votes18.csv",index_label=["CandidateID","County"])